## 2024年 訪日外客の消費単価　地域別データの可視化

## データについて
利用するデータのファイルは２種類
1. foreign_visitors_2024.csv: 訪日外客数
1. inbound_consumption_2024.csv: 消費単価

### 訪日外客数
* 出典：日本政府観光局「JNTO日本の観光統計データ」国籍/目的別 訪日外客数（2004年～2024年）（PDF）
* URL: https://www.jnto.go.jp/statistics/data/visitors-statistics/
* JNTOから外客数データを取得　時系列推移表　国籍/月別 訪日外客数（2004年～2024年）（PDF）からファイルを取得。2024年の観光客などの人数を抽出・加工しました。


### 消費単価
* 出典：「インバウンド消費動向調査（旧 訪日外国人消費動向調査）」（観光庁）
* 「インバウンド消費動向調査（旧 訪日外国人消費動向調査）『2024年年間集計表』」（観光庁）(https://www.mlit.go.jp/kankocho/tokei_hakusyo/gaikokujinshohidoko.html)から抜粋
* 「居住地（21区分）別　費目別購入率および購入者単価」を加工して作成

### 国・地域コード
* ISO3コードを利用する。
* [wiki iso3-code](https://ja.wikipedia.org/wiki/ISO_3166-1)を参考に国・地域名に対応するコードを追記してあります。

In [1]:
import plotly.express as px
import pandas as pd

In [2]:
df = pd.read_csv("./data/inbound_consumption_2024.csv")
df = df.dropna()   # その他の行を削除
df

,地域,コード,人数,消費単価
0,韓国,KOR,3529,98385.2324
1,台湾,TWN,2898,154042.4972
2,香港,HKG,2835,230987.6834
3,中国,CHN,4326,256701.8395
4,タイ,THA,1374,174018.8578
5,シンガポール,SGP,677,280167.2053
6,マレーシア,MYS,1452,182498.6922
7,インドネシア,IDN,890,190815.9443
8,フィリピン,PHL,981,156029.0050
9,ベトナム,VNM,1426,187013.9071


### コロプレスマップ 
* 国・地域コードを利用して訪日観光客の居住地を塗り分ける
* データセットのコード列で塗り分け
* 人数列をrange_colorの幅でグラデーション
* natural earth図を利用

In [3]:
# 1. データの読み込み
df = pd.read_csv("./data/inbound_consumption_2024.csv")
df = df.dropna()   # その他の行を削除

# 2. 最小値と最大値を利用してグラデーションの範囲を決める。
# 下限：最小値x0.9
# 上限：最大値x1.1
margin = 0.1
min_value = df["消費単価"].min()*(1-margin)
max_value = df["消費単価"].max()*(1+margin)

# 2. 地図表示
fig = px.choropleth(df, 
                    locations="コード",
                    color="消費単価",
                    hover_name="地域",
                    hover_data={"消費単価": ":,",       # ツールチップの設定。カンマ区切り表示
                                "コード": False},       # 「コード」は非表示
                    range_color=[min_value, max_value], # 色の範囲を指定
                    color_continuous_scale="Rainbow")    # グラデーションの色指定

# 3. 地図の基本設定
fig.update_geos(projection_type="natural earth",          # 図法
                landcolor="rgba(255, 255, 255, 0.1)",   # 地図の基本色
                projection_rotation=dict(lat=0, lon=148), # 日本を中心に持ってくる
                showcountries=True,                       # 国境描画
                countrycolor="lightgray",                 # 国境線の色。
                countrywidth=0.3,                         # 国境線の太さ。デフォルト：0.5        
                )

# 4. レイアウトに関する設定
fig.update_layout(dragmode=False,          # マウスの利用不可
                  coloraxis_colorbar=dict(title="消費単価（円）",
                                          tickformat=",.0f"  # カンマ区切り整数
                  ),
                  width=800,
                  height=550,
                  title="2024年　訪日観光客の消費単価")
# 5. 表示
fig.show()
